In [1]:
%matplotlib inline

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [9]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = "cpu"

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [14]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [27]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
epochs = 5
for i in range(epochs):
    print(f"Epoch {i+1}\n--------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------
Loss: 2.176623 [    0/60000]
Loss: 2.173571 [ 6400/60000]
Loss: 2.113327 [12800/60000]
Loss: 2.138641 [19200/60000]
Loss: 2.089816 [25600/60000]
Loss: 2.032600 [32000/60000]
Loss: 2.070633 [38400/60000]
Loss: 1.983275 [44800/60000]
Loss: 1.997515 [51200/60000]
Loss: 1.936207 [57600/60000]
Test Error: 
 Accuracy: 59.1%, Avg loss: 1.924630 

Epoch 2
--------------
Loss: 1.955193 [    0/60000]
Loss: 1.935366 [ 6400/60000]
Loss: 1.812721 [12800/60000]
Loss: 1.859259 [19200/60000]
Loss: 1.752578 [25600/60000]
Loss: 1.699788 [32000/60000]
Loss: 1.731497 [38400/60000]
Loss: 1.614089 [44800/60000]
Loss: 1.653850 [51200/60000]
Loss: 1.545423 [57600/60000]
Test Error: 
 Accuracy: 61.5%, Avg loss: 1.552538 

Epoch 3
--------------
Loss: 1.620713 [    0/60000]
Loss: 1.590237 [ 6400/60000]
Loss: 1.425395 [12800/60000]
Loss: 1.504451 [19200/60000]
Loss: 1.377413 [25600/60000]
Loss: 1.370164 [32000/60000]
Loss: 1.392339 [38400/60000]
Loss: 1.295908 [44800/60000]
Loss: 1.350019 

In [30]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
